In [ ]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import IPython.display as ipd
from scipy.io import wavfile
import glob
import os
import json
import shutil

# 各種設定
pd.options.display.max_columns=None
%matplotlib inline

In [ ]:
# confirm the number of filess

audio_path = "./train/audio/"
view1_path = "./train/view1/rgb/"
view2_path = "./train/view2/rgb/"
view3_path = "./train/view3/rgb/"
view4_path = "./train/view4/rgb/"

# audio
print("audio ; ", sum(os.path.isfile(os.path.join(audio_path,name)) for name in os.listdir(audio_path)))

# view 
print("view1 ; ", sum(os.path.isfile(os.path.join(view1_path,name)) for name in os.listdir(view1_path)))
print("view2 ; ", sum(os.path.isfile(os.path.join(view2_path,name)) for name in os.listdir(view2_path)))
print("view3 ; ", sum(os.path.isfile(os.path.join(view3_path,name)) for name in os.listdir(view3_path)))
print("view4 ; ", sum(os.path.isfile(os.path.join(view4_path,name)) for name in os.listdir(view4_path)))



# 注釈ファイル(jsonファイル)の読み込み

In [ ]:
open_file = open("annotation.json")

# jsonファイルとして読み込む
annotation_json = json.load(open_file)
# 正しく読み込めているか確認
print (len(annotation_json["annotations"]))

In [ ]:
# 最初の10このデータの確認
for i in range(10):
    print('*'*25, 'data{}'.format(i), '*'*25)
    print(annotation_json['annotations'][i])
    print()

# pandasを使ってjsonファイルを読み込む

In [ ]:
df = pd.DataFrame(annotation_json["annotations"])
df.head()

In [ ]:
folder = "./train/audio/"
files = glob.glob(folder+"*.wav")

files_view1 = sorted(glob.glob("./train/view1/rgb/*.mp4"))
files_view2 = sorted(glob.glob("./train/view2/rgb/*.mp4"))
files_view3 = sorted(glob.glob("./train/view3/rgb/*.mp4"))
files_view4 = sorted(glob.glob("./train/view4/rgb/*.mp4"))

files = sorted(files)

df["audio_file"] = files
df["rgb_view1"] = files_view1
df["rgb_view2"] = files_view2
df["rgb_view3"] = files_view3
df["rgb_view4"] = files_view4

df = df.drop(index=377)
print(df.shape)
df.head()

In [ ]:
df = df.sort_values("container id")

# 列名 の変更
columns = ["id", "container_id", "scenario", "background", "illumination", "width_at_the_top", "width_at_the_bottom", \
    "height", "depth", "container_capacity", "container_mass", "filling_type", "filling_level", "filling_density", "filling_mass", \
        "object_mass", "handover_starting_frame", "handover_start_timestamp", "handover_hand", "audio_file", \
            "rgb_view1", "rgb_view2", "rgb_view3", "rgb_view4"]
df.columns = columns
print(df.shape)
df.head()

# filling type のデータの個数の可視化
> id=0 : none  
> id=1 : pasta  
> id=2 : rice  
> id=3 : water

In [ ]:
plt.title('Count by filling_type')
df['filling_type'].value_counts().plot.bar()

# filling_levelごとのデータの個数の可視化
> id=0 : empty(0%)  
> id=1 : half-full(50%)  
> id=2 : full(90%)

In [ ]:
df['filling_level'].value_counts().plot.bar()
plt.title('Count by filling_level')

In [ ]:
# # 移動前のフォルダのpath
# DIR_BEFORE = "./train/"
# # 移動後のファイルpath
# DIR_AFTER = "./data/"

# # 音声ファイルをフォルダへ移す
# # フォルダの作成
# for i in range(1, 10):
#     os.makedirs("./data/{}/".format(i), exist_ok=True)
#     os.makedirs(f"./data/{i}/audio", exist_ok=True)
#     os.makedirs(f"./data/{i}/view1/rgb/", exist_ok=True)
#     os.makedirs(f"./data/{i}/view2/rgb/", exist_ok=True)
#     os.makedirs(f"./data/{i}/view3/rgb/", exist_ok=True)
#     os.makedirs(f"./data/{i}/view4/rgb/", exist_ok=True)

# # 1行ずつ処理
# for i in range(df.shape[0]):
#     row = df.iloc[i, :]
#     # 各ファイルの移動前のpath
#     audio_b = row["audio_file"]
#     view1_b = row["rgb_view1"]
#     view2_b = row["rgb_view2"]
#     view3_b = row["rgb_view3"]
#     view4_b = row["rgb_view4"]

#     # 音声ファイルの名前の変更 
#     s = row["scenario"]
#     fi = row["filling_type"]
#     fu = row["filling_level"]
#     b = row["background"]
#     l = row["illumination"]
#     c = 2
#     # audio_af = f"s{s}_fi{fi}_fu{fu}_b{b}_l{l}_c{c}_audio.wav"
#     audio_af = f"s{s}_fi{fi}_fu{fu}_b{b}_l{l}_audio.wav"
#     os.rename(audio_b, DIR_BEFORE+"audio/"+audio_af)
    
#     # # 移動後のファイル名
#     # view1_af = row["rgb_view1"].split("/")[-1]
#     # view2_af = row["rgb_view2"].split("/")[-1]
#     # view3_af = row["rgb_view3"].split("/")[-1]
#     # view4_af = row["rgb_view4"].split("/")[-1]
    
#     # 移動後のファイルpath
#     audio_bf = DIR_BEFORE + "audio/" + audio_af
#     after_dir = DIR_AFTER+str(row["container_id"])+"/"

#     # 各ファイルを移動
#     shutil.move(audio_bf, after_dir+"audio/")
#     shutil.move(view1_b, after_dir+"view1/rgb/")
#     shutil.move(view2_b, after_dir+"view2/rgb/")
#     shutil.move(view3_b, after_dir+"view3/rgb/")
#     shutil.move(view4_b, after_dir+"view4/rgb/")

In [ ]:
columns = ["folder_num", "num", "file_name", "subject", "filling_type", "filling_level", "back", "light", \
    "camera_id", "start", "end"]
df_csv = pd.DataFrame(index=None, columns=columns)

row_temp = {"folder_num":[], "num":[], "file_name":[], "subject":[], "filling_type":[],
                "filling_level":[], "back":[], "light":[], "camera_id":[], "start":[], "end":[]}

# folder_numの設定
for i in range(1, 10):
    count = 0

    for df_fol in df[df["container_id"] == i].itertuples():
        fname = f"s{df_fol.scenario}_fi{df_fol.filling_type}_fu{df_fol.filling_level}" + \
                f"_b{df_fol.background}_l{df_fol.illumination}_c2"
        row_temp["folder_num"].append(i)
        row_temp["num"].append(count)
        row_temp["file_name"].append(fname)
        row_temp["subject"].append(df_fol.scenario)
        row_temp["filling_type"].append(df_fol.filling_type)
        row_temp["filling_level"].append(df_fol.filling_level)
        row_temp["back"].append(df_fol.background)
        row_temp["camera_id"].append(2)
        row_temp["light"].append(df_fol.illumination)
        row_temp["start"].append(df_fol.handover_start_timestamp)
        row_temp["end"].append(-1)

        count += 1

df_csv = pd.concat([df_csv, pd.DataFrame.from_dict(row_temp)])

print(df_csv.shape)
df_csv.head()

In [ ]:
df_sort = df_csv.sort_values(["folder_num", "file_name"])
df_sort.head()

In [ ]:
df_by_fname = pd.DataFrame(index=None, columns=columns)
for i in range(1, 10):
    num = 0
    df_by_folder = df_sort[df_sort["folder_num"] == i]
    for j in range(0, df_by_folder.shape[0]):
        df_by_folder.iat[j, 1] = num
        num += 1
    df_by_fname = pd.concat([df_by_fname, df_by_folder], axis=0)
df_by_fname.head()

In [ ]:
# # save csv file
# df_by_fname.to_csv("annotations_sort.csv", index=False)

In [ ]:
# # save csv file
# df_csv.to_csv("annotations.csv", index=False)

In [ ]:
df["container_id"].value_counts()

In [ ]:
df_sort["folder_num"].value_counts()

In [ ]:
"""
# 移動前のフォルダのpath
DIR_BEFORE = "./train/"
# 移動後のファイルpath
DIR_AFTER = "./data/"

# 音声ファイルをフォルダへ移す
# フォルダの作成
for i in range(1, 10):
    os.makedirs("./data/{}/".format(i), exist_ok=True)
    os.makedirs(f"./data/{i}/audio", exist_ok=True)
    os.makedirs(f"./data/{i}/view1/rgb/", exist_ok=True)
    os.makedirs(f"./data/{i}/view2/rgb/", exist_ok=True)
    os.makedirs(f"./data/{i}/view3/rgb/", exist_ok=True)
    os.makedirs(f"./data/{i}/view4/rgb/", exist_ok=True)

# 1行ずつ処理
for i in range(df.shape[0]):
    row = df.iloc[i, :]
    # 各ファイルの移動前のpath
    audio_b = row["audio_file"]
    view1_b = row["rgb_view1"]
    view2_b = row["rgb_view2"]
    view3_b = row["rgb_view3"]
    view4_b = row["rgb_view4"]

    # 音声ファイルの名前の変更 
    s = row["scenario"]
    fi = row["filling_type"]
    fu = row["filling_level"]
    b = row["background"]
    l = row["illumination"]
    c = 2
    # audio_af = f"s{s}_fi{fi}_fu{fu}_b{b}_l{l}_c{c}_audio.wav"
    audio_af = f"s{s}_fi{fi}_fu{fu}_b{b}_l{l}_audio.wav"
    os.rename(audio_b, DIR_BEFORE+"audio/"+audio_af)
"""